In [ ]:
import torch
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_transform = transforms.Compose([
    transforms.Resize(232),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(232),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def evaluate_attack(weights_path=os.path.join(os.getcwd(), "outputs/model_best.pth")):
    model = models.resnet18(pretrained=False)

    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    if not os.path.exists(weights_path):
        print("ERROR: Model weights not found. Train first.")
        return
    
    model.load_state_dict(torch.load(weights_path))
    model.to(DEVICE)
    model.eval()

    